# CS182 Final Project

### Named-Entity Recognition using an HMM Model

I think that building the tagger will have a large enough scope, especially when you show different approaches/ideas and try to maximize your scores. I am also happy to discuss in person early next week. 

Regarding your ideas: I am not sure why you want a full Bayes-Net - I believe a HMM+Viterbi is sufficient for the task so I recommend implementing that. Here are slides from CS287 that talk about NER and how to approach the problem: https://cs287.github.io/Lectures/slides/lecture14-search.pdf
It could be useful for you to compare the performance between the forward and viterbi algorithm here. 

When you consider more information, please be wary that HMMs/Bayes-Nets have the Markov assumption. You need to be careful when taking previous and following words into account that you don’t violate that assumption. Depending on the size of your corpus, cross-validation might also not be necessary and a simple train/valid/test split could be sufficient.

In [10]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [11]:
data = pd.read_csv("entity-annotated-corpus/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
data.head()

Skipping line 281837: expected 25 fields, saw 34



,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [12]:
# drop null rows and check if any null values remaining
data.dropna(inplace=True)
data[data.isnull().any(axis=1)].size

0

In [13]:
# SOME EDA
print("Possible predictive data: " + str(list(set(data.columns.values))))
print
print("What do all of these mean?")
print
print("Tags: " + str(list(set(data.tag))))
print
print("What do all of these mean?")
print
print("Length of data set: " + str(len(data)))

Possible predictive data: [u'pos', u'shape', u'tag', u'prev-lemma', u'prev-prev-lemma', u'next-word', u'next-lemma', 'Unnamed: 0', u'next-next-word', u'lemma', u'prev-prev-iob', u'sentence_idx', u'next-next-pos', u'prev-iob', u'next-shape', u'prev-shape', u'prev-prev-word', u'next-next-shape', u'next-pos', u'prev-prev-pos', u'word', u'prev-pos', u'prev-prev-shape', u'prev-word', u'next-next-lemma']

What do all of these mean?

Tags: [u'I-art', u'B-gpe', u'B-art', u'I-per', u'I-tim', u'B-org', u'O', u'B-geo', u'B-tim', u'I-geo', u'B-per', u'I-eve', u'B-eve', u'I-gpe', u'I-org', u'I-nat', u'B-nat']

What do all of these mean?

Length of data set: 1050794


In [53]:
data_small = data[:100000]
data_valid = data[100001:150000]
preds = list(data.columns.values)
preds.remove('tag')
y_small = data_small['tag']
x_small = data_small[preds]

# Split into train and test data

x_train, x_test, y_train, y_test = train_test_split(x_small, y_small, test_size=0.2, random_state=0)
print(x_train.shape)
print(y_train.shape)

(80000, 24)
(80000,)


In [54]:
pos_list = list(set(x_train['pos']))
print(pos_list)
print
shape_list = list(set(x_train['shape']))
print(shape_list)
print
tag_list = list(set(y_train.values))
print(tag_list)


[u'PRP$', u'VBG', u'VBD', u'``', u'VBN', u'POS', u'VBP', u'WDT', u'JJ', u'WP', u'VBZ', u'DT', u'RP', u'$', u'NN', u',', u'.', u'TO', u'PRP', u'RB', u';', u':', u'NNS', u'NNP', u'VB', u'WRB', u'RRB', u'CC', u'PDT', u'RBS', u'RBR', u'CD', u'LRB', u'EX', u'IN', u'WP$', u'MD', u'NNPS', u'JJS', u'JJR', u'UH']

[u'mixedcase', u'lowercase', u'camelcase', u'ending-dot', u'capitalized', u'number', u'abbreviation', u'punct', u'other', u'uppercase', u'contains-hyphen']

[u'I-art', u'B-gpe', u'B-art', u'I-per', u'I-tim', u'B-org', u'O', u'B-geo', u'B-tim', u'I-geo', u'B-per', u'I-eve', u'B-eve', u'I-gpe', u'I-org', u'I-nat', u'B-nat']


In [55]:

# build a dict linking shape to likelihood of each tag

shape_probs = {}
pos_probs = {}

# if X shape, then what is the most likely tag?

for shape in shape_list:
    tag_prob_list = []
    for tag in tag_list:
        count = 0
        for i in data_small[data_small['shape'] == shape]['tag']:
            if i == tag:
                count += 1
        tag_prob_list.append(1.0*count/len(data_small[data_small['shape'] == shape]))
    index = tag_prob_list.index(max(tag_prob_list))
        
    shape_probs[shape] = tag_list[index]
    
for pos in pos_list:
    tag_prob_list = []
    for tag in tag_list:
        count = 0
        for i in data_small[data_small['shape'] == pos]['pos']:
            if i == tag:
                count += 1
        tag_prob_list.append(1.0*count/(len(data_small[data_small['shape'] == pos])+1))
    index = tag_prob_list.index(max(tag_prob_list))
        
    pos_probs[pos] = tag_list[index]

print(shape_probs)
print(pos_probs)
        

{u'mixedcase': u'O', u'lowercase': u'O', u'camelcase': u'I-per', u'ending-dot': u'B-per', u'number': u'O', u'capitalized': u'O', u'abbreviation': u'B-geo', u'punct': u'O', u'other': u'O', u'uppercase': u'O', u'contains-hyphen': u'O'}
{u'PRP$': u'I-art', u'VBG': u'I-art', u'VBD': u'I-art', u'VBN': u'I-art', u',': u'I-art', u'VBP': u'I-art', u'WDT': u'I-art', u'JJ': u'I-art', u'WP': u'I-art', u'VBZ': u'I-art', u'DT': u'I-art', u'RP': u'I-art', u'$': u'I-art', u'NN': u'I-art', u'POS': u'I-art', u'.': u'I-art', u'TO': u'I-art', u'PRP': u'I-art', u'RB': u'I-art', u';': u'I-art', u':': u'I-art', u'NNS': u'I-art', u'NNP': u'I-art', u'``': u'I-art', u'WRB': u'I-art', u'RRB': u'I-art', u'CC': u'I-art', u'PDT': u'I-art', u'RBS': u'I-art', u'RBR': u'I-art', u'CD': u'I-art', u'NNPS': u'I-art', u'EX': u'I-art', u'IN': u'I-art', u'WP$': u'I-art', u'MD': u'I-art', u'LRB': u'I-art', u'JJS': u'I-art', u'JJR': u'I-art', u'VB': u'I-art', u'UH': u'I-art'}


In [58]:
pred_train = []
pred_valid = []

num_O = len(data[data['tag'] == 'O'])
percent = 1.0*num_O/len(data)
print "Percent of data without a tag (Baseline accuracy if we only predict 'O': " + str(percent)

# training prediction
count_correct = 0
for i in range(len(data_small)):
    pred_tag = shape_probs[data_small.iloc[i]['shape']]
    pred_train.append(pred_tag)
    if data_small.iloc[i]['tag'] == pred_tag:
        count_correct += 1
train_accuracy = 1.0*count_correct / len(data_small)
print "Train Accuracy: " + str(train_accuracy)

# validation prediction
count_correct = 0
for i in range(len(data_valid)):
    pred_tag = shape_probs[data_valid.iloc[i]['shape']]
    pred_train.append(pred_tag)
    if data_valid.iloc[i]['tag'] == pred_tag:
        count_correct += 1
train_accuracy = 1.0*count_correct / len(data_valid)
print "Validation Accuracy: " + str(train_accuracy)

Percent of data without a tag (Baseline accuracy if we only predict 'O': 0.846952875635
Train Accuracy: 0.853
Validation Accuracy: 0.854017080342
